In [648]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import requests as req
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import * 
import time
import json

In [649]:
options = Options()
options.add_argument("--start-maximized")

In [650]:
web_drive_path = r"C:\Users\chaud\Desktop\GoogleChromeDriver\chromedriver.exe"

In [651]:
website_url = "https://www.naukri.com/"

In [652]:
keyword = 'python'

In [653]:
driver = webdriver.Chrome(web_drive_path, options=options)

In [654]:
driver.get(website_url)

In [655]:
wait = WebDriverWait(driver, 7)# wait till 7 seconds

In [656]:
searchbox= wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="root"]/div[7]/div/div/div[1]/div/div/div/div[1]/div/input')))
searchbox.send_keys(keyword)

In [657]:
submit = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="root"]/div[7]/div/div/div[6]')))
submit.click()

time.sleep(2)

In [658]:
freshness_Job = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="search-result-container"]/div[1]/div[1]/div/div/div[2]/div[14]/div[2]/div')))
driver.execute_script("arguments[0].scrollIntoView();", freshness_Job)
freshness_Job.click()


In [659]:
select_last_7_days = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="search-result-container"]/div[1]/div[1]/div/div/div[2]/div[14]/div[2]/div/div/ul/li[3]')))
select_last_7_days.click()
time.sleep(2)

In [667]:
temp_data = []

In [668]:
while True:
    
    feed_list = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "srp-jobtuple-wrapper")))
    main_page_script = WebDriverWait(driver, 7).until(EC.presence_of_element_located((By.XPATH, '//*[@id="search-result-container"]/div[1]/script[2]')))
    main_page_script_pointers = json.loads(main_page_script.get_attribute('innerHTML'))['itemListElement']
    
    job_days = WebDriverWait(driver, 7).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'job-post-day')))
    job_days_text = [elem.text for elem in job_days]
    
    for i in range(len(feed_list)):
        if "Today" in job_days_text[i] or "Just Now" in job_days_text[i] or "Few Hours Ago" in job_days_text[i] or "Few Minutes Ago" in job_days_text[i]:
            num_days = 0  # Consider these as 0 days old
        else:
            # Try to extract the number of days from the string
            try:
                num_days = int(job_days_text[i].split()[0])  # Extract the number from the text
            except ValueError:
                # In case there's any other unexpected format, set num_days to 0 or skip
                continue
        
        if num_days > 7:
            break
                
        else:
            # Store the current window handle
            main_window = driver.current_window_handle
            try:
                # Click on the job listing to open in a new tab
                feed_list[i].click()
                time.sleep(3)

                # Get all the window handles
                windows = driver.window_handles
                # Switch to the new tab (the last opened window)
                driver.switch_to.window(windows[-1])

                try:
                    # Extract the job details from the script element
                    job_link = WebDriverWait(driver, 7).until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="root"]/div/script[1]')))
                    job_data_json = json.loads(job_link[0].get_attribute('innerHTML'))

                    # Initialize empty dictionary to store job data
                    job_data = {}

                    # Keys to extract from JSON obje

                    # Append the complete job_data dictionary to temp_data list
                    job_data['URL'] = main_page_script_pointers[i].get('url', 'None')  # Fallback URL
                    job_data['name'] = main_page_script_pointers[i].get('name', 'Unknown Job')
                    
                    temp_data.append(job_data)

                except TimeoutException:
                    print(f"Error extracting job data for listing {i}.")

                # Close the new tab
                driver.close()
                # Switch back to the main window
                driver.switch_to.window(main_window)

            except Exception as e:
                # Handle cases where the job feed click or tab handling fails
                print(f"Error processing job listing {i}: {e}")
                continue  # Continue to the next listing

    next_button = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="lastCompMark"]/a[2]')))
    if next_button.get_attribute('disabled'):
        print("Reached the last page.")
        break
    next_button.click()
    time.sleep(2)


Error extracting job data for listing 16.
Error extracting job data for listing 2.
Error extracting job data for listing 3.
Error extracting job data for listing 15.
Error extracting job data for listing 17.
Error extracting job data for listing 18.
Error extracting job data for listing 19.
Error extracting job data for listing 4.
Error extracting job data for listing 9.
Error extracting job data for listing 10.
Error extracting job data for listing 19.
Error extracting job data for listing 9.
Error extracting job data for listing 10.
Error extracting job data for listing 4.
Error extracting job data for listing 6.
Error extracting job data for listing 11.
Error extracting job data for listing 3.
Error extracting job data for listing 5.
Error extracting job data for listing 12.
Error extracting job data for listing 13.
Error extracting job data for listing 17.
Error extracting job data for listing 4.
Error extracting job data for listing 18.
Error extracting job data for listing 8.
Err

TimeoutException: Message: 


In [680]:
file_Name = f"{keyword}-Naurki.csv"
pd.DataFrame(temp_data).to_csv(file_Name)